# Summary 
Generates the geojson hexes+dissolved cities file.

Demonstrates calculation of 50/10 access levels from the NBD map data against arbitrary 
geometry other than the precomputed hexagons.

Calculating this requires joining the geometry from the Canada wide hexagons geometry 
with the tabular `PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv` dataset. 

Loading the data and executing the spatial joins takes a total of 7 minutes.

In [1]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

/Users/zacharyshand/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

from src.datasets import joins
import src

In [3]:
popctrs = statcan.boundary('population_centres')

In [4]:
popctrs.loc[lambda s:s.PCNAME == "Lloydminster"]

,PCUID,PCNAME,PCTYPE,PCPUID,PCCLASS,PRUID,PRNAME,CMAUID,CMANAME,CMATYPE,CMAPUID,geometry
657,0478,Lloydminster,1,480478,3,48,Alberta,840,Lloydminster (Alberta part / partie de l'Alberta),D,48840,"POLYGON ((5024336.049 2072898.043, 5024239.789..."
774,0478,Lloydminster,1,470478,3,47,Saskatchewan,840,Lloydminster (Saskatchewan part / partie de la...,D,47840,"POLYGON ((5024135.374 2071102.100, 5024109.260..."


In [5]:
custom_areas = joins.hexagons_small_popctrs_combined()
o = custom_areas

/Users/zacharyshand/ookla-statcan-analysis/notebooks/../src/datasets/overlays.py:25: UserWarning: `keep_geom_type=True` in overlay resulted in 418 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ol = gp.overlay(left, right, how="union")
/Users/zacharyshand/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
phh_speed_csv = src.config.DATA_DIRECTORY / 'PHH' / 'CRTC_Speed_Data' / 'PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv'
phh_speeds = pd.read_csv(phh_speed_csv)

phh_geo = statcan.phh_geometry()

phh = phh_geo.merge(phh_speeds, on='PHH_ID')
phh = phh.merge(statcan.phh_data(), on='PHH_ID')
phh = phh.loc[lambda s:s.Type !=8] # drop PHH nulls

In [7]:
o = joins.add_phh_pop(custom_areas, phh, 'HEXUID_PCPUID')

In [8]:
del o['HEXuid_HEXidu'] # we don't want these, they're quite long in large cities

In [9]:
popctrs = statcan.boundary('population_centres') # use this to generate a filtering region around cities

In [10]:
#Can't plot the whole country, too many hexagons.
xmin, ymin, xmax, ymax = popctrs.loc[lambda s:s.PCNAME=="Lloydminster"].buffer(100000).total_bounds
o.cx[xmin:xmax,ymin:ymax].loc[lambda s:(s['Pop2016'] > 0) | (s['TDwell2016_TLog2016'] > 0) | (s['URDwell2016_RH2016']>0)].explore('Pop_Avail_50_10',  scheme='equalinterval', k = 4,)

In [11]:
zeroable_cols = [
    'Pop2016', 'TDwell2016_TLog2016', 'URDwell2016_RH2016',
    'Pop2016_at_50_10_Combined', 'TDwell2016_at_50_10_Combined',
    'URDwell_at_50_10_Combined'
       ]
o[zeroable_cols] = o[zeroable_cols].fillna(0)

In [12]:
o

,PCPUID,pc_area,hex_area,geometry,hex_frac,pc_frac,HEXUID_PCPUID,PRCODE,PCNAME,PCCLASS,...,TDwell2016_TLog2016,URDwell2016_RH2016,PHH_Count,Common_Type,Pop2016_at_50_10_Combined,TDwell2016_at_50_10_Combined,URDwell_at_50_10_Combined,Pop_Avail_50_10,TDwell_Avail_50_10,URDwell_Avail_50_10
55,100792,1.719773e+08,2.539316e+07,"MULTIPOLYGON (((8975148.894 2149398.714, 89753...",0.440441,0.065033,NL47580528-100792,NL,St. John's,4,...,5058.332094,4318.056742,685.0,3.0,9860.925340,5043.554316,4304.723408,99.644081,99.707853,99.691219
56,100792,1.719773e+08,2.539316e+07,"POLYGON ((8967674.386 2143964.411, 8967679.029...",0.514429,0.075958,NL47580529-100792,NL,St. John's,4,...,1648.265873,1558.259524,437.0,4.0,4252.819048,1648.265873,1558.259524,100.000000,100.000000,100.000000
57,100792,1.719773e+08,2.536795e+07,"POLYGON ((8971687.209 2162844.838, 8971644.644...",0.000288,0.000043,NL47710527-100792,NL,St. John's,4,...,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
58,100792,1.719773e+08,2.542434e+07,"POLYGON ((8983681.000 2140410.863, 8983745.474...",0.043914,0.006492,NL47430528-100792,NL,St. John's,4,...,1056.284918,1025.692964,149.0,3.0,2662.527342,1056.284918,1025.692964,100.000000,100.000000,100.000000
59,100792,1.719773e+08,2.540903e+07,"POLYGON ((8979409.323 2142110.794, 8979381.251...",0.619943,0.091594,NL47510528-100792,NL,St. John's,4,...,10449.110393,9692.839561,1684.0,3.0,24029.650382,10449.110393,9692.839561,100.000000,100.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519965,NaN,NaN,2.441105e+07,"POLYGON ((4391568.918 2319505.032, 4389028.114...",1.000000,NaN,BC53331204,BC,NaN,NaN,...,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,NaN,NaN,NaN
519966,NaN,NaN,2.447289e+07,"POLYGON ((4370293.149 2275226.474, 4367751.202...",1.000000,NaN,BC52891204,BC,NaN,NaN,...,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
519967,NaN,NaN,2.450451e+07,"POLYGON ((4359727.416 2253237.353, 4357184.726...",1.000000,NaN,BC52661204,BC,NaN,NaN,...,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN
519968,NaN,NaN,2.444205e+07,"POLYGON ((4380906.610 2297314.924, 4378365.292...",1.000000,NaN,BC53111204,BC,NaN,NaN,...,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [13]:
from pathlib import Path
from src import config

In [14]:
derived_geometry = (Path(src.config.DATA_DIRECTORY) / 'processed' / 'geometries').resolve()
derived_geometry.mkdir(exist_ok=True, parents=True)

o.to_crs("EPSG:4326").to_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs.geojson', driver='GeoJSON')

In [16]:
o.to_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs', driver="MapInfo File")